In [14]:
from pandas import read_csv
from statsmodels.tsa.arima_model import ARIMA
import numpy

# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        #print(dataset[i])
        diff.append(value)
    return numpy.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

series = read_csv('/Users/parnabbasak/Documents/workspace/Group3_Hackathon/Node_Services/uploads/sales_history.csv')
#Filter dataframe by product
filter_by_product_series = series[series['producct']=="Brand A1-Phone"]
#print(filter_by_product_series)

#Filter dataframe by attribute
filter_by_product_and_aiitribute_series = filter_by_product_series[filter_by_product_series['attribute']=="Net Unit"]
#print(filter_by_product_and_aiitribute_series)


X = filter_by_product_and_aiitribute_series["quantity"].tolist()
#print(X)

weeks_in_year = 52
differenced = difference(X, weeks_in_year)
# fit model
model = ARIMA(differenced, order=(1,1,1))
model_fit = model.fit(disp=0)
# multi-step out-of-sample forecast
forecast = model_fit.forecast(steps=52)[0]
# invert the differenced forecast to something usable
history = [x for x in X]
#history
week = 1
predictions = []
for yhat in forecast:
    inverted = inverse_difference(history, yhat, weeks_in_year)
    print('Week %d: %f' % (week, inverted))
    history.append(inverted)
    predictions.append(inverted)
    week += 1

print(predictions)


Week 1: 109.000000
Week 2: 117.000000
Week 3: 111.000000
Week 4: 112.000000
Week 5: 113.000000
Week 6: 114.000000
Week 7: 115.000000
Week 8: 116.000000
Week 9: 117.000000
Week 10: 118.000000
Week 11: 119.000000
Week 12: 120.000000
Week 13: 121.000000
Week 14: 122.000000
Week 15: 123.000000
Week 16: 124.000000
Week 17: 125.000000
Week 18: 126.000000
Week 19: 127.000000
Week 20: 128.000000
Week 21: 129.000000
Week 22: 130.000000
Week 23: 131.000000
Week 24: 132.000000
Week 25: 133.000000
Week 26: 134.000000
Week 27: 135.000000
Week 28: 136.000000
Week 29: 137.000000
Week 30: 138.000000
Week 31: 139.000000
Week 32: 140.000000
Week 33: 141.000000
Week 34: 142.000000
Week 35: 143.000000
Week 36: 144.000000
Week 37: 145.000000
Week 38: 146.000000
Week 39: 147.000000
Week 40: 148.000000
Week 41: 149.000000
Week 42: 150.000000
Week 43: 151.000000
Week 44: 152.000000
Week 45: 153.000000
Week 46: 154.000000
Week 47: 155.000000
Week 48: 156.000000
Week 49: 157.000000
Week 50: 158.000000
Week 51: 

/usr/local/lib/python2.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [70]:
from datetime import datetime, timedelta
 
date_str = '2020-01-01'
date_obj = datetime.strptime(date_str, '%Y-%m-%d')
 
start_of_week = date_obj - timedelta(days=date_obj.weekday())  # Monday
end_of_week = start_of_week + timedelta(days=6)  # Sunday
print(start_of_week)
print(end_of_week)

2019-12-30 00:00:00
2020-01-05 00:00:00


In [6]:
import pandas as pd
from datetime import datetime, timedelta
import calendar
import datetime
from pymongo import MongoClient
#############################################################
#This is a simple dummy program created to learn more about python based iteration and handling
# of data. Does not have any PII business data or business logic copied.
#############################################################
#Establisth MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client.Forecast

forecasts = db.salesforecasts

#Delete all existing Data in Forecast Collection
#x = forecasts.delete_many({})
#print(x.deleted_count, " documents deleted.")

#############################################################
# This function calculates all the week start dates of an year
#futureYearWeekStarts = []
futureYearWeekStarts = ['06/Jan/2020', '13/Jan/2020', '20/Jan/2020', '27/Jan/2020', '03/Feb/2020', '10/Feb/2020', '17/Feb/2020', '24/Feb/2020', '02/Mar/2020', '09/Mar/2020', '16/Mar/2020', '23/Mar/2020', '30/Mar/2020', '06/Apr/2020', '13/Apr/2020', '20/Apr/2020', '27/Apr/2020', '04/May/2020', '11/May/2020', '18/May/2020', '25/May/2020', '01/Jun/2020', '08/Jun/2020', '15/Jun/2020', '22/Jun/2020', '29/Jun/2020', '06/Jul/2020', '13/Jul/2020', '20/Jul/2020', '27/Jul/2020', '03/Aug/2020', '10/Aug/2020', '17/Aug/2020', '24/Aug/2020', '31/Aug/2020', '07/Sep/2020', '14/Sep/2020', '21/Sep/2020', '28/Sep/2020', '05/Oct/2020', '12/Oct/2020', '19/Oct/2020', '26/Oct/2020', '02/Nov/2020', '09/Nov/2020', '16/Nov/2020', '23/Nov/2020', '30/Nov/2020', '07/Dec/2020', '14/Dec/2020', '21/Dec/2020', '28/Dec/2020']
#def get_start_and_end_date_from_calendar_week(year, calendar_week):       
#    monday = datetime.datetime.strptime(f'{year}-{calendar_week}-1', "%Y-%W-%w").date()
#    return monday.strftime('%d/%b/%Y')

#for x in range(1, 53):
#   futureYearWeekStarts.append(get_start_and_end_date_from_calendar_week(2020, x))

#print(futureYearWeekStarts)
#############################################################
# This function creates individual docs to be inserted in the collection
def merge(list1, list2, prod, chn, attr): 
    merged_list = []  
    for i in range(0, len(list1)):
        doc_obj = {"channel": chn,
                 "saleDate": list1[i],
                 "producct": prod,
                 "attribute": attr,
                 "quantity": list2[i]}
        merged_list.append(doc_obj)  
    return merged_list 

#############################################################

data = {
        'channel': ['Channel1', 'Channel1', 'Channel1', 'Channel2', 'Channel2'], 
        'producct': ['productA', 'productB', 'productA', 'productA', 'productA'], 
        'attribute': ['NET', 'NET', 'NET', 'NET', 'otherd'],
        'quantity': [1, 2, 3, 4, 5]
       }

df = pd.DataFrame(data)

#Find unique channels
uniquechannels = df['channel'].unique()

#Iterating over unique channels
for i in uniquechannels: 
    
    #Get data for every unique channels 
    on_channel = df[df['channel'] == i]
    
    #Find unique products for each channel
    uniqueproducts = on_channel.producct.unique()
    
    #Iterating over every unique product for a specific channel
    for j in uniqueproducts:
        
        #Get data for every unique product for a specific channel
        on_channel_and_product = on_channel[on_channel['producct'] == j]
        
        #Only select attribute = NET since that would be used for ARIMA calculation
        on_channel_and_product_attribute = on_channel_and_product[on_channel_and_product['attribute'] == 'NET']
        #print(on_channel_and_product_attribute)
        
        #This is a dummy prediction
        predictions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 
                         27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52 ]
        
        merged_forecast = merge(futureYearWeekStarts, predictions, j, i, 'ATTRIB')
        
        x = forecasts.insert_many(merged_forecast)
        print(merged_forecast)
        print('----Inserted x.inserted_ids elements in the  collection------------------------------------------------------')
        

[{'attribute': 'ATTRIB', 'producct': 'productA', 'saleDate': '06/Jan/2020', '_id': ObjectId('5dc3e3d314abcebce50f4e5f'), 'channel': 'Channel1', 'quantity': 1}, {'attribute': 'ATTRIB', 'producct': 'productA', 'saleDate': '13/Jan/2020', '_id': ObjectId('5dc3e3d314abcebce50f4e60'), 'channel': 'Channel1', 'quantity': 2}, {'attribute': 'ATTRIB', 'producct': 'productA', 'saleDate': '20/Jan/2020', '_id': ObjectId('5dc3e3d314abcebce50f4e61'), 'channel': 'Channel1', 'quantity': 3}, {'attribute': 'ATTRIB', 'producct': 'productA', 'saleDate': '27/Jan/2020', '_id': ObjectId('5dc3e3d314abcebce50f4e62'), 'channel': 'Channel1', 'quantity': 4}, {'attribute': 'ATTRIB', 'producct': 'productA', 'saleDate': '03/Feb/2020', '_id': ObjectId('5dc3e3d314abcebce50f4e63'), 'channel': 'Channel1', 'quantity': 5}, {'attribute': 'ATTRIB', 'producct': 'productA', 'saleDate': '10/Feb/2020', '_id': ObjectId('5dc3e3d314abcebce50f4e64'), 'channel': 'Channel1', 'quantity': 6}, {'attribute': 'ATTRIB', 'producct': 'productA

In [6]:
import pandas as pd
from datetime import datetime, timedelta
import calendar
import datetime
from pymongo import MongoClient
#############################################################
#This is a simple dummy program created to learn more about python based iteration and handling
# of data. Does not have any PII business data or business logic copied.
#############################################################
#Establisth MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client.Forecast

saleDate = date_time_obj = datetime.datetime.strptime('06/Jan/2020', '%d/%b/%Y')
print(saleDate)

forecasts = db.salesforecasts

forecasts.insert_one({'attribute': 'ATTRIB', 'producct': 'productA', 'saleDate': saleDate,'channel': 'Channel1', 'quantity': 1})
        

2020-01-06 00:00:00
